# load pkgs

In [1]:
import sys
sys.path.insert(0, '..')

In [2]:
import PipelineTS
PipelineTS.__version__

'0.3.10'

In [3]:
from PipelineTS.dataset import LoadMessagesSentDataSets
from PipelineTS.io import save_model, load_model

In [4]:
import pandas as pd
time_col = 'date'
target_col = 'ta'

init_data = LoadMessagesSentDataSets()[[time_col, target_col]]
init_data[time_col] = pd.to_datetime(init_data[time_col])

In [5]:
accelerator = 'auto'
lags = 60
n = 40
quantile = 0.9

In [6]:
valid_data = init_data.iloc[-n:, :]
init_series = init_data.iloc[-lags * 3:-lags, :]
test_data = init_data.iloc[-lags:, :]
train_data = init_data.iloc[:-n, :]

train_data.tail(5)

,date,ta
557,2021-07-11,125957
558,2021-07-12,139012
559,2021-07-13,127014
560,2021-07-14,118241
561,2021-07-15,114358


# Preprocessing

In [7]:
from PipelineTS.metrics import quantile_acc
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
from PipelineTS.spinesTS.metrics import wmape

from PipelineTS.preprocessing import Scaler

In [8]:
# use a scaler to scale data
data = train_data.copy()

series = init_series.copy()

scaler = Scaler(scaler_name='min_max')

data[target_col] = scaler.fit_transform(train_data[target_col].values.reshape(-1, 1)).squeeze()
valid_data[target_col] = scaler.transform(valid_data[target_col].values.reshape(-1, 1)).squeeze()
series[target_col] = scaler.fit_transform(series[target_col].values.reshape(-1, 1)).squeeze()
series.tail()

,date,ta
537,2021-06-21,0.784289
538,2021-06-22,0.679875
539,2021-06-23,0.281006
540,2021-06-24,0.412787
541,2021-06-25,0.712538


In [9]:
def inverse_transform_prediction(df, scaler):
    """define a function to inverse data."""
    for i in df.columns:
        if i != time_col:
            df[i] = scaler.inverse_transform(df[i].values.reshape(-1, 1)).squeeze()

In [10]:
def prediction_metrics(true_df, pred_df):
    """define a function to evaluate the performance of model."""
    yt = true_df[target_col].values
    res = {}
    for i in pred_df.columns:
        if i == target_col:
            res['mae'] = mean_absolute_error(yt.reshape(-1, 1), pred_df[i].values.reshape(-1, 1))
            res['mape'] = mean_absolute_percentage_error(yt.reshape(-1, 1), pred_df[i].values.reshape(-1, 1))
            res['wmape'] = wmape(yt.reshape(-1, 1), pred_df[i].values.reshape(-1, 1))
            
    if target_col+'_lower' in pred_df.columns and target_col+'_upper' in pred_df.columns:
        res['quantile_acc'] = quantile_acc(yt, pred_df[target_col+'_lower'].values, pred_df[target_col+'_upper'].values)

    return res

In [11]:
def test_load_and_save_model(fp, model, scaler, data):
    save_model(fp, model, scaler=scaler)
    model, scaler = load_model(fp)
    res = model.predict(test_data.shape[0], data=data)
    inverse_transform_prediction(res, scaler)
    print(prediction_metrics(test_data, res))
    return res

# WideGBRTModel

In [12]:
from PipelineTS.ml_model import WideGBRTModel

model = WideGBRTModel(time_col=time_col, target_col=target_col, lags=lags, random_state=42, 
                          quantile=quantile, n_jobs=-1)
model.fit(data)

In [13]:
res = model.predict(test_data.shape[0], data=series)
inverse_transform_prediction(res, scaler)
print(prediction_metrics(test_data, res))
res

{'mae': 23274.60747020986, 'mape': 0.18274336298428562, 'wmape': 0.17722695461372648, 'quantile_acc': 0.23333333333333334}


,date,ta,ta_lower,ta_upper
0,2021-06-26,134770.196928,126654.714656,142885.679201
1,2021-06-27,137164.662697,128494.721118,145834.604276
2,2021-06-28,142353.893156,132482.340278,152225.446033
3,2021-06-29,142164.904431,132337.113531,151992.695331
4,2021-06-30,142022.199011,132227.452789,151816.945234
5,2021-07-01,133944.957090,126020.566256,141869.347925
6,2021-07-02,137447.553883,128712.106398,146183.001369
7,2021-07-03,138582.683422,129584.386851,147580.979993
8,2021-07-04,135191.661504,126978.585455,143404.737553
9,2021-07-05,135650.376507,127331.081023,143969.671992


In [14]:
test_load_and_save_model('wide_gbrt.zip', model, scaler, data=series)

{'mae': 23274.60747020986, 'mape': 0.18274336298428562, 'wmape': 0.17722695461372648, 'quantile_acc': 0.23333333333333334}


,date,ta,ta_lower,ta_upper
0,2021-06-26,134770.196928,126654.714656,142885.679201
1,2021-06-27,137164.662697,128494.721118,145834.604276
2,2021-06-28,142353.893156,132482.340278,152225.446033
3,2021-06-29,142164.904431,132337.113531,151992.695331
4,2021-06-30,142022.199011,132227.452789,151816.945234
5,2021-07-01,133944.957090,126020.566256,141869.347925
6,2021-07-02,137447.553883,128712.106398,146183.001369
7,2021-07-03,138582.683422,129584.386851,147580.979993
8,2021-07-04,135191.661504,126978.585455,143404.737553
9,2021-07-05,135650.376507,127331.081023,143969.671992


# GBDT

In [15]:
from PipelineTS.ml_model import *

## CatBoost

In [16]:
model = CatBoostModel(time_col=time_col, target_col=target_col, lags=30, random_state=42, quantile=quantile)
model.fit(data=data)

In [17]:
res = model.predict(test_data.shape[0], data=series)
inverse_transform_prediction(res, scaler)
print(prediction_metrics(test_data, res))
res

{'mae': 22206.983868955384, 'mape': 0.16541922172904003, 'wmape': 0.16909742204196346, 'quantile_acc': 0.16666666666666666}


,date,ta,ta_lower,ta_upper
0,2021-06-26,148098.128425,136902.802344,159293.454506
1,2021-06-27,134758.280023,126650.159239,142866.400806
2,2021-06-28,139002.666744,129912.278938,148093.054550
3,2021-06-29,137250.590845,128565.681260,145935.500430
4,2021-06-30,141271.843812,131656.306450,150887.381174
5,2021-07-01,138904.561840,129836.878189,147972.245492
6,2021-07-02,130697.420603,123529.093611,137865.747594
7,2021-07-03,132087.134987,124597.190127,139577.079848
8,2021-07-04,129564.954322,122658.710961,136471.197683
9,2021-07-05,129261.301778,122425.331910,136097.271646


In [18]:
test_load_and_save_model('catboost.zip', model, scaler, data=series)

{'mae': 22206.983868955384, 'mape': 0.16541922172904003, 'wmape': 0.16909742204196346, 'quantile_acc': 0.16666666666666666}


,date,ta,ta_lower,ta_upper
0,2021-06-26,148098.128425,136902.802344,159293.454506
1,2021-06-27,134758.280023,126650.159239,142866.400806
2,2021-06-28,139002.666744,129912.278938,148093.054550
3,2021-06-29,137250.590845,128565.681260,145935.500430
4,2021-06-30,141271.843812,131656.306450,150887.381174
5,2021-07-01,138904.561840,129836.878189,147972.245492
6,2021-07-02,130697.420603,123529.093611,137865.747594
7,2021-07-03,132087.134987,124597.190127,139577.079848
8,2021-07-04,129564.954322,122658.710961,136471.197683
9,2021-07-05,129261.301778,122425.331910,136097.271646


## XGBoost

In [19]:
model = XGBoostModel(time_col=time_col, target_col=target_col, lags=lags, random_state=42, quantile=quantile)
model.fit(data=data)

In [20]:
res = model.predict(test_data.shape[0], data=series)
inverse_transform_prediction(res, scaler)
print(prediction_metrics(test_data, res))
res

{'mae': 21995.497916666667, 'mape': 0.15871965431014692, 'wmape': 0.167487039941398, 'quantile_acc': 0.18333333333333332}


,date,ta,ta_lower,ta_upper
0,2021-06-26,150294.390625,138406.703125,162182.078125
1,2021-06-27,132702.156250,124949.835938,140454.500000
2,2021-06-28,144724.953125,134146.437500,155303.437500
3,2021-06-29,143798.687500,133437.921875,154159.437500
4,2021-06-30,153023.281250,140494.109375,165552.437500
5,2021-07-01,138549.140625,129422.375000,147675.890625
6,2021-07-02,132803.218750,125027.132812,140579.296875
7,2021-07-03,126200.789062,119976.726562,132424.859375
8,2021-07-04,130498.117188,123263.882812,137732.343750
9,2021-07-05,125432.695312,119389.179688,131476.203125


In [21]:
test_load_and_save_model('xgboost.zip', model, scaler, data=series)

{'mae': 21995.497916666667, 'mape': 0.15871965431014692, 'wmape': 0.167487039941398, 'quantile_acc': 0.18333333333333332}


,date,ta,ta_lower,ta_upper
0,2021-06-26,150294.390625,138406.703125,162182.078125
1,2021-06-27,132702.156250,124949.835938,140454.500000
2,2021-06-28,144724.953125,134146.437500,155303.437500
3,2021-06-29,143798.687500,133437.921875,154159.437500
4,2021-06-30,153023.281250,140494.109375,165552.437500
5,2021-07-01,138549.140625,129422.375000,147675.890625
6,2021-07-02,132803.218750,125027.132812,140579.296875
7,2021-07-03,126200.789062,119976.726562,132424.859375
8,2021-07-04,130498.117188,123263.882812,137732.343750
9,2021-07-05,125432.695312,119389.179688,131476.203125


## LightGBM

In [22]:
model = LightGBMModel(time_col=time_col, target_col=target_col, lags=n, random_state=42, quantile=quantile, verbose=-1)
model.fit(data=data)

In [23]:
res = model.predict(test_data.shape[0], data=series)
inverse_transform_prediction(res, scaler)
print(prediction_metrics(test_data, res))
res

{'mae': 22544.07025487705, 'mape': 0.15736542954700533, 'wmape': 0.1716642018982987, 'quantile_acc': 0.13333333333333333}


,date,ta,ta_lower,ta_upper
0,2021-06-26,142369.417900,131273.964754,153464.871045
1,2021-06-27,138080.660328,128101.026776,148060.293880
2,2021-06-28,157534.013175,142493.139414,172574.886935
3,2021-06-29,151949.855518,138361.829679,165537.881356
4,2021-06-30,131936.300374,123555.264344,140317.336404
5,2021-07-01,119854.393090,114616.745011,125092.041169
6,2021-07-02,130338.000573,122372.799247,138303.201899
7,2021-07-03,123276.890325,117148.800338,129404.980311
8,2021-07-04,129411.850619,121687.608649,137136.092588
9,2021-07-05,121151.184344,115576.145992,126726.222696


In [24]:
test_load_and_save_model('lightgbm.zip', model, scaler, data=series)

{'mae': 22544.07025487705, 'mape': 0.15736542954700533, 'wmape': 0.1716642018982987, 'quantile_acc': 0.13333333333333333}


,date,ta,ta_lower,ta_upper
0,2021-06-26,142369.417900,131273.964754,153464.871045
1,2021-06-27,138080.660328,128101.026776,148060.293880
2,2021-06-28,157534.013175,142493.139414,172574.886935
3,2021-06-29,151949.855518,138361.829679,165537.881356
4,2021-06-30,131936.300374,123555.264344,140317.336404
5,2021-07-01,119854.393090,114616.745011,125092.041169
6,2021-07-02,130338.000573,122372.799247,138303.201899
7,2021-07-03,123276.890325,117148.800338,129404.980311
8,2021-07-04,129411.850619,121687.608649,137136.092588
9,2021-07-05,121151.184344,115576.145992,126726.222696


## RandomForest

In [25]:
model = RandomForestModel(time_col=time_col, target_col=target_col, lags=lags, random_state=42, quantile=quantile)
model.fit(data=data)

In [26]:
res = model.predict(test_data.shape[0], data=series)
inverse_transform_prediction(res, scaler)
print(prediction_metrics(test_data, res))
res

{'mae': 21953.641577125603, 'mape': 0.1547890965396645, 'wmape': 0.16716832042711036, 'quantile_acc': 0.11666666666666667}


,date,ta,ta_lower,ta_upper
0,2021-06-26,127711.456164,121836.103819,133586.808509
1,2021-06-27,126304.453175,120724.459683,131884.446667
2,2021-06-28,125573.446379,120146.906242,130999.986516
3,2021-06-29,124858.622921,119582.138923,130135.106918
4,2021-06-30,124304.171002,119144.077861,129464.264143
5,2021-07-01,123972.586296,118882.099601,129063.072990
6,2021-07-02,124020.823763,118920.211032,129121.436494
7,2021-07-03,123315.146373,118362.669834,128267.622912
8,2021-07-04,122671.176238,117853.882270,127488.470207
9,2021-07-05,122900.072149,118034.728223,127765.416075


In [27]:
test_load_and_save_model('rf.zip', model, scaler, data=series)

{'mae': 21953.641577125603, 'mape': 0.1547890965396645, 'wmape': 0.16716832042711036, 'quantile_acc': 0.11666666666666667}


,date,ta,ta_lower,ta_upper
0,2021-06-26,127711.456164,121836.103819,133586.808509
1,2021-06-27,126304.453175,120724.459683,131884.446667
2,2021-06-28,125573.446379,120146.906242,130999.986516
3,2021-06-29,124858.622921,119582.138923,130135.106918
4,2021-06-30,124304.171002,119144.077861,129464.264143
5,2021-07-01,123972.586296,118882.099601,129063.072990
6,2021-07-02,124020.823763,118920.211032,129121.436494
7,2021-07-03,123315.146373,118362.669834,128267.622912
8,2021-07-04,122671.176238,117853.882270,127488.470207
9,2021-07-05,122900.072149,118034.728223,127765.416075


# NN Model

In [28]:
from PipelineTS.nn_model import *

## DLinearModel

In [29]:
model = DLinearModel(time_col=time_col, target_col=target_col, lags=10, random_state=42, quantile=quantile)

In [30]:
model.fit(data)

In [31]:
res = model.predict(test_data.shape[0], data=series)
inverse_transform_prediction(res, scaler)
print(prediction_metrics(test_data, res))
res

{'mae': 22261.970052083332, 'mape': 0.1742332436956994, 'wmape': 0.16951612013575826, 'quantile_acc': 0.23333333333333334}


,date,ta,ta_lower,ta_upper
0,2021-06-26,168721.328125,154500.343750,182942.312500
1,2021-06-27,165110.203125,151633.484375,178586.937500
2,2021-06-28,182937.515625,165786.484375,200088.546875
3,2021-06-29,175638.234375,159991.625000,191284.859375
4,2021-06-30,151081.234375,140495.968750,161666.484375
5,2021-07-01,154282.453125,143037.406250,165527.515625
6,2021-07-02,168171.109375,154063.515625,182278.687500
7,2021-07-03,129960.984375,123728.750000,136193.234375
8,2021-07-04,138209.359375,130277.085938,146141.640625
9,2021-07-05,152469.468750,141598.093750,163340.843750


In [32]:
test_load_and_save_model('dlinear.zip', model, scaler, data=series)

{'mae': 22261.970052083332, 'mape': 0.1742332436956994, 'wmape': 0.16951612013575826, 'quantile_acc': 0.23333333333333334}


,date,ta,ta_lower,ta_upper
0,2021-06-26,168721.328125,154500.343750,182942.312500
1,2021-06-27,165110.203125,151633.484375,178586.937500
2,2021-06-28,182937.515625,165786.484375,200088.546875
3,2021-06-29,175638.234375,159991.625000,191284.859375
4,2021-06-30,151081.234375,140495.968750,161666.484375
5,2021-07-01,154282.453125,143037.406250,165527.515625
6,2021-07-02,168171.109375,154063.515625,182278.687500
7,2021-07-03,129960.984375,123728.750000,136193.234375
8,2021-07-04,138209.359375,130277.085938,146141.640625
9,2021-07-05,152469.468750,141598.093750,163340.843750


##  NLinearModel

In [33]:
model = NLinearModel(time_col=time_col, target_col=target_col, lags=lags, random_state=42, quantile=quantile)

In [34]:
model.fit(data)
res = model.predict(test_data.shape[0], data=series)
inverse_transform_prediction(res, scaler)
print(prediction_metrics(test_data, res))
res

{'mae': 25465.108072916668, 'mape': 0.2072321461978939, 'wmape': 0.193906752603624, 'quantile_acc': 0.21666666666666667}


,date,ta,ta_lower,ta_upper
0,2021-06-26,164308.718750,147142.968750,181474.484375
1,2021-06-27,174605.171875,154702.812500,194507.546875
2,2021-06-28,174953.265625,154958.390625,194948.156250
3,2021-06-29,174583.593750,154686.953125,194480.218750
4,2021-06-30,174749.734375,154808.937500,194690.515625
5,2021-07-01,173310.656250,153752.359375,192868.968750
6,2021-07-02,166689.390625,148890.906250,184487.875000
7,2021-07-03,157016.531250,141788.906250,172244.140625
8,2021-07-04,152520.937500,138488.171875,166553.703125
9,2021-07-05,148890.093750,135822.343750,161957.859375


In [35]:
test_load_and_save_model('nlinear.zip', model, scaler, data=series)

{'mae': 25465.108072916668, 'mape': 0.2072321461978939, 'wmape': 0.193906752603624, 'quantile_acc': 0.21666666666666667}


,date,ta,ta_lower,ta_upper
0,2021-06-26,164308.718750,147142.968750,181474.484375
1,2021-06-27,174605.171875,154702.812500,194507.546875
2,2021-06-28,174953.265625,154958.390625,194948.156250
3,2021-06-29,174583.593750,154686.953125,194480.218750
4,2021-06-30,174749.734375,154808.937500,194690.515625
5,2021-07-01,173310.656250,153752.359375,192868.968750
6,2021-07-02,166689.390625,148890.906250,184487.875000
7,2021-07-03,157016.531250,141788.906250,172244.140625
8,2021-07-04,152520.937500,138488.171875,166553.703125
9,2021-07-05,148890.093750,135822.343750,161957.859375


##  NBeatsModel 

In [36]:
model = NBeatsModel(time_col=time_col, target_col=target_col, lags=lags, random_state=42, quantile=quantile)

In [37]:
model.fit(data)
res = model.predict(test_data.shape[0], data=series)
inverse_transform_prediction(res, scaler)
print(prediction_metrics(test_data, res))
res

{'mae': 24260.872135416666, 'mape': 0.1760831598327671, 'wmape': 0.18473697098162672, 'quantile_acc': 0.18333333333333332}


,date,ta,ta_lower,ta_upper
0,2021-06-26,174290.125000,156124.156250,192456.093750
1,2021-06-27,154035.390625,140803.843750,167266.921875
2,2021-06-28,121206.570312,115972.757812,126440.375000
3,2021-06-29,131004.453125,123383.687500,138625.203125
4,2021-06-30,157578.078125,143483.484375,171672.687500
5,2021-07-01,142168.906250,131828.265625,152509.531250
6,2021-07-02,134721.781250,126195.398438,143248.140625
7,2021-07-03,130586.796875,123067.789062,138105.812500
8,2021-07-04,118392.617188,113844.335938,122940.882812
9,2021-07-05,115291.515625,111498.726562,119084.296875


In [38]:
test_load_and_save_model('nbeats.zip', model, scaler, data=series)

{'mae': 24260.872135416666, 'mape': 0.1760831598327671, 'wmape': 0.18473697098162672, 'quantile_acc': 0.18333333333333332}


,date,ta,ta_lower,ta_upper
0,2021-06-26,174290.125000,156124.156250,192456.093750
1,2021-06-27,154035.390625,140803.843750,167266.921875
2,2021-06-28,121206.570312,115972.757812,126440.375000
3,2021-06-29,131004.453125,123383.687500,138625.203125
4,2021-06-30,157578.078125,143483.484375,171672.687500
5,2021-07-01,142168.906250,131828.265625,152509.531250
6,2021-07-02,134721.781250,126195.398438,143248.140625
7,2021-07-03,130586.796875,123067.789062,138105.812500
8,2021-07-04,118392.617188,113844.335938,122940.882812
9,2021-07-05,115291.515625,111498.726562,119084.296875


## TCNModel

In [39]:
model = TCNModel(time_col=time_col, target_col=target_col, lags=lags, random_state=42, quantile=quantile)

In [40]:
model.fit(data)
res = model.predict(test_data.shape[0], data=series)
inverse_transform_prediction(res, scaler)
print(prediction_metrics(test_data, res))
res

{'mae': 29963.155843285283, 'mape': 0.25447711970501014, 'wmape': 0.22815761200350135, 'quantile_acc': 0.2833333333333333}


,date,ta,ta_lower,ta_upper
0,2021-06-26,147945.001699,135918.387413,159971.615985
1,2021-06-27,151596.999691,138659.573542,164534.425840
2,2021-06-28,141170.001341,130833.078756,151506.923926
3,2021-06-29,122399.001003,116743.584222,128054.417783
4,2021-06-30,141084.998968,130769.276058,151400.721878
5,2021-07-01,128410.000485,121255.434567,135564.566403
6,2021-07-02,128003.000244,120949.940583,135056.059905
7,2021-07-03,132020.000431,123965.097003,140074.903859
8,2021-07-04,139727.001963,129749.964832,149704.039095
9,2021-07-05,147330.999641,135457.518067,159204.481214


In [41]:
test_load_and_save_model('tcn.zip', model, scaler, data=series)

{'mae': 29963.155843285283, 'mape': 0.25447711970501014, 'wmape': 0.22815761200350135, 'quantile_acc': 0.2833333333333333}


,date,ta,ta_lower,ta_upper
0,2021-06-26,147945.001699,135918.387413,159971.615985
1,2021-06-27,151596.999691,138659.573542,164534.425840
2,2021-06-28,141170.001341,130833.078756,151506.923926
3,2021-06-29,122399.001003,116743.584222,128054.417783
4,2021-06-30,141084.998968,130769.276058,151400.721878
5,2021-07-01,128410.000485,121255.434567,135564.566403
6,2021-07-02,128003.000244,120949.940583,135056.059905
7,2021-07-03,132020.000431,123965.097003,140074.903859
8,2021-07-04,139727.001963,129749.964832,149704.039095
9,2021-07-05,147330.999641,135457.518067,159204.481214


## NHitsModel

In [42]:
model = NHitsModel(time_col=time_col, target_col=target_col, lags=lags, random_state=42, quantile=quantile)

In [43]:
model.fit(data)
res = model.predict(test_data.shape[0], data=series)
inverse_transform_prediction(res, scaler)
print(prediction_metrics(test_data, res))
res

{'mae': 26512.918880208334, 'mape': 0.20650118269950832, 'wmape': 0.20188541856503012, 'quantile_acc': 0.16666666666666666}


,date,ta,ta_lower,ta_upper
0,2021-06-26,171319.140625,155071.046875,187567.250000
1,2021-06-27,170667.734375,154567.468750,186768.031250
2,2021-06-28,170367.484375,154335.359375,186399.625000
3,2021-06-29,174427.984375,157474.359375,191381.625000
4,2021-06-30,162083.812500,147931.593750,176236.046875
5,2021-07-01,148708.015625,137591.296875,159824.718750
6,2021-07-02,139993.000000,130854.093750,149131.937500
7,2021-07-03,135055.015625,127036.726562,143073.296875
8,2021-07-04,133113.640625,125535.929688,140691.343750
9,2021-07-05,135046.515625,127030.164062,143062.875000


In [44]:
test_load_and_save_model('nhits.zip', model, scaler, data=series)

{'mae': 26512.918880208334, 'mape': 0.20650118269950832, 'wmape': 0.20188541856503012, 'quantile_acc': 0.16666666666666666}


,date,ta,ta_lower,ta_upper
0,2021-06-26,171319.140625,155071.046875,187567.250000
1,2021-06-27,170667.734375,154567.468750,186768.031250
2,2021-06-28,170367.484375,154335.359375,186399.625000
3,2021-06-29,174427.984375,157474.359375,191381.625000
4,2021-06-30,162083.812500,147931.593750,176236.046875
5,2021-07-01,148708.015625,137591.296875,159824.718750
6,2021-07-02,139993.000000,130854.093750,149131.937500
7,2021-07-03,135055.015625,127036.726562,143073.296875
8,2021-07-04,133113.640625,125535.929688,140691.343750
9,2021-07-05,135046.515625,127030.164062,143062.875000


## TFTModel

In [45]:
model = TFTModel(time_col=time_col, target_col=target_col, lags=lags, random_state=42, quantile=quantile, accelerator='cpu')
model.fit(data)

In [46]:
res = model.predict(test_data.shape[0], data=series)
inverse_transform_prediction(res, scaler)
print(prediction_metrics(test_data, res))
res

{'mae': 22189.829296875, 'mape': 0.15429459078293045, 'wmape': 0.16896679674263673, 'quantile_acc': 0.2}


,date,ta,ta_lower,ta_upper
0,2021-07-16,113934.257812,110410.179688,117458.351562
1,2021-07-17,114657.515625,110954.085938,118360.953125
2,2021-07-18,115208.789062,111368.648438,119048.921875
3,2021-07-19,115540.640625,111618.218750,119463.070312
4,2021-07-20,115764.406250,111786.492188,119742.312500
5,2021-07-21,115942.218750,111920.210938,119964.234375
6,2021-07-22,116102.078125,112040.421875,120163.726562
7,2021-07-23,116256.062500,112156.226562,120355.906250
8,2021-07-24,116408.968750,112271.210938,120546.710938
9,2021-07-25,116562.250000,112386.484375,120738.015625


In [47]:
test_load_and_save_model('tft.zip', model, scaler, data=series)

{'mae': 22189.829296875, 'mape': 0.15429459078293045, 'wmape': 0.16896679674263673, 'quantile_acc': 0.2}


,date,ta,ta_lower,ta_upper
0,2021-07-16,113934.257812,110410.179688,117458.351562
1,2021-07-17,114657.515625,110954.085938,118360.953125
2,2021-07-18,115208.789062,111368.648438,119048.921875
3,2021-07-19,115540.640625,111618.218750,119463.070312
4,2021-07-20,115764.406250,111786.492188,119742.312500
5,2021-07-21,115942.218750,111920.210938,119964.234375
6,2021-07-22,116102.078125,112040.421875,120163.726562
7,2021-07-23,116256.062500,112156.226562,120355.906250
8,2021-07-24,116408.968750,112271.210938,120546.710938
9,2021-07-25,116562.250000,112386.484375,120738.015625


## GAU

In [48]:
from PipelineTS.nn_model import *

In [49]:
model = GAUModel(time_col=time_col, target_col='ta', lags=lags, random_state=42, 
                     quantile=quantile, accelerator=accelerator, patience=10, verbose=True)
model.fit(data)

MPS  available: True   | MPS  use: True
CUDA available: False  | CUDA use: False
TPU  available: False  | TPU  use: False
CPU  available: True   | CPU  use: False


Epoch 1/1000  
1/1 - p0 -  [*lr: 0.0009045] - loss: 0.6981 - mae: 0.6981 - val_loss: 0.6322 - val_mae: 0.6322 - 3.17s/epoch - 3.172s/step
Epoch 2/1000  
1/1 - p0 -  [*lr: 0.0006545] - loss: 0.6322 - mae: 0.6322 - val_loss: 0.5778 - val_mae: 0.5778 - 0.04s/epoch - 0.036s/step
Epoch 3/1000  
1/1 - p0 -  [*lr: 0.0003455] - loss: 0.5778 - mae: 0.5778 - val_loss: 0.5411 - val_mae: 0.5411 - 0.03s/epoch - 0.031s/step
Epoch 4/1000  
1/1 - p0 -  [*lr: 9.55e-05] - loss: 0.5411 - mae: 0.5411 - val_loss: 0.5221 - val_mae: 0.5221 - 0.03s/epoch - 0.031s/step
Epoch 5/1000  
1/1 - p0 - [*lr: 0.000000e+00] - loss: 0.5221 - mae: 0.5221 - val_loss: 0.5168 - val_mae: 0.5168 - 0.03s/epoch - 0.030s/step
Epoch 6/1000  
1/1 - p1 -  [*lr: 9.55e-05] - loss: 0.5168 - mae: 0.5168 - val_loss: 0.5168 - val_mae: 0.5168 - 0.03s/epoch - 0.030s/step
Epoch 7/1000  
1/1 - p0 -  [*lr: 0.0003455] - loss: 0.5168 - mae: 0.5168 - val_loss: 0.5115 - val_mae: 0.5115 - 0.03s/epoch - 0.031s/step
Epoch 8/1000  
1/1 - p0 -  [*lr: 0

In [50]:
res = model.predict(test_data.shape[0], data=series)
inverse_transform_prediction(res, scaler)
print(prediction_metrics(test_data, res))
res

{'mae': 24534.85920577993, 'mape': 0.17953500375568726, 'wmape': 0.18682327444114452, 'quantile_acc': 0.23333333333333334}


,date,ta,ta_lower,ta_upper
0,2021-06-26,125164.138467,117849.776893,132478.500041
1,2021-06-27,113165.226154,109300.568031,117029.884277
2,2021-06-28,98091.407059,98560.492227,97622.321891
3,2021-06-29,108281.360209,105820.820123,110741.900296
4,2021-06-30,114211.663432,110046.153182,118377.173682
5,2021-07-01,116148.134270,111425.886045,120870.382494
6,2021-07-02,111409.270225,108049.451796,114769.088654
7,2021-07-03,116629.304143,111768.718932,121489.889353
8,2021-07-04,164212.940007,145671.995399,182753.884615
9,2021-07-05,155280.719892,139307.800597,171253.639187


In [51]:
test_load_and_save_model('gau.zip', model, scaler, data=series)

{'mae': 24534.85920577993, 'mape': 0.17953500375568726, 'wmape': 0.18682327444114452, 'quantile_acc': 0.23333333333333334}


,date,ta,ta_lower,ta_upper
0,2021-06-26,125164.138467,117849.776893,132478.500041
1,2021-06-27,113165.226154,109300.568031,117029.884277
2,2021-06-28,98091.407059,98560.492227,97622.321891
3,2021-06-29,108281.360209,105820.820123,110741.900296
4,2021-06-30,114211.663432,110046.153182,118377.173682
5,2021-07-01,116148.134270,111425.886045,120870.382494
6,2021-07-02,111409.270225,108049.451796,114769.088654
7,2021-07-03,116629.304143,111768.718932,121489.889353
8,2021-07-04,164212.940007,145671.995399,182753.884615
9,2021-07-05,155280.719892,139307.800597,171253.639187


## StackingRNN

In [52]:
model = StackingRNNModel(time_col=time_col, target_col=target_col, lags=10, random_state=42, 
                     quantile=quantile, accelerator=accelerator, patience=100, verbose=True)
model.fit(data)

MPS  available: True   | MPS  use: True
CUDA available: False  | CUDA use: False
TPU  available: False  | TPU  use: False
CPU  available: True   | CPU  use: False


Epoch 1/1000  
1/1 - p0 -  [*lr: 0.0009045] - loss: 0.1735 - mae: 0.1735 - val_loss: 0.1254 - val_mae: 0.1254 - 2.05s/epoch - 2.053s/step
Epoch 2/1000  
1/1 - p0 -  [*lr: 0.0006545] - loss: 0.1254 - mae: 0.1254 - val_loss: 0.0876 - val_mae: 0.0876 - 0.02s/epoch - 0.025s/step
Epoch 3/1000  
1/1 - p0 -  [*lr: 0.0003455] - loss: 0.0876 - mae: 0.0876 - val_loss: 0.0672 - val_mae: 0.0672 - 0.03s/epoch - 0.025s/step
Epoch 4/1000  
1/1 - p0 -  [*lr: 9.55e-05] - loss: 0.0672 - mae: 0.0672 - val_loss: 0.0595 - val_mae: 0.0595 - 0.02s/epoch - 0.024s/step
Epoch 5/1000  
1/1 - p0 - [*lr: 0.000000e+00] - loss: 0.0595 - mae: 0.0595 - val_loss: 0.0577 - val_mae: 0.0577 - 0.02s/epoch - 0.024s/step
Epoch 6/1000  
1/1 - p1 -  [*lr: 9.55e-05] - loss: 0.0577 - mae: 0.0577 - val_loss: 0.0577 - val_mae: 0.0577 - 0.02s/epoch - 0.023s/step
Epoch 7/1000  
1/1 - p0 -  [*lr: 0.0003455] - loss: 0.0577 - mae: 0.0577 - val_loss: 0.0561 - val_mae: 0.0561 - 0.02s/epoch - 0.024s/step
Epoch 8/1000  
1/1 - p0 -  [*lr: 0

In [53]:
res = model.predict(test_data.shape[0], data=series)
inverse_transform_prediction(res, scaler)
print(prediction_metrics(test_data, res))
res

{'mae': 20974.814231494565, 'mape': 0.1568745459776819, 'wmape': 0.1597149363139362, 'quantile_acc': 0.11666666666666667}


,date,ta,ta_lower,ta_upper
0,2021-06-26,154842.610138,145320.449309,164364.770968
1,2021-06-27,160485.499315,149988.503972,170982.494657
2,2021-06-28,155380.694758,145765.577388,164995.812129
3,2021-06-29,150101.982058,141398.786074,158805.178042
4,2021-06-30,151614.418909,142649.942649,160578.895168
5,2021-07-01,145409.834018,137517.227735,153302.440301
6,2021-07-02,143939.916613,136301.245180,151578.588047
7,2021-07-03,143873.262645,136246.105985,151500.419305
8,2021-07-04,141755.746369,134494.400179,149017.092558
9,2021-07-05,140630.951015,133563.918288,147697.983743


In [54]:
test_load_and_save_model('stacking_rnn.zip', model, scaler, data=series)

{'mae': 20974.814231494565, 'mape': 0.1568745459776819, 'wmape': 0.1597149363139362, 'quantile_acc': 0.11666666666666667}


,date,ta,ta_lower,ta_upper
0,2021-06-26,154842.610138,145320.449309,164364.770968
1,2021-06-27,160485.499315,149988.503972,170982.494657
2,2021-06-28,155380.694758,145765.577388,164995.812129
3,2021-06-29,150101.982058,141398.786074,158805.178042
4,2021-06-30,151614.418909,142649.942649,160578.895168
5,2021-07-01,145409.834018,137517.227735,153302.440301
6,2021-07-02,143939.916613,136301.245180,151578.588047
7,2021-07-03,143873.262645,136246.105985,151500.419305
8,2021-07-04,141755.746369,134494.400179,149017.092558
9,2021-07-05,140630.951015,133563.918288,147697.983743


## Time2Vec

In [56]:
model = Time2VecModel(time_col=time_col, target_col=target_col, lags=lags, random_state=42, 
                     quantile=quantile, accelerator=accelerator, patience=10, verbose=True)
model.fit(data)

MPS  available: True   | MPS  use: True
CUDA available: False  | CUDA use: False
TPU  available: False  | TPU  use: False
CPU  available: True   | CPU  use: False


Epoch 1/1000  
1/1 - p0 -  [*lr: 0.0090451] - loss: 0.1461 - mae: 0.1461 - val_loss: 0.1178 - val_mae: 0.1178 - 1.02s/epoch - 1.016s/step
Epoch 2/1000  
1/1 - p0 -  [*lr: 0.0065451] - loss: 0.1178 - mae: 0.1178 - val_loss: 0.0724 - val_mae: 0.0724 - 0.02s/epoch - 0.022s/step
Epoch 3/1000  
1/1 - p0 -  [*lr: 0.0034549] - loss: 0.0724 - mae: 0.0724 - val_loss: 0.0599 - val_mae: 0.0599 - 0.02s/epoch - 0.020s/step
Epoch 4/1000  
1/1 - p0 -  [*lr: 0.0009549] - loss: 0.0599 - mae: 0.0599 - val_loss: 0.0545 - val_mae: 0.0545 - 0.02s/epoch - 0.019s/step
Epoch 5/1000  
1/1 - p0 - [*lr: 0.000000e+00] - loss: 0.0545 - mae: 0.0545 - val_loss: 0.0527 - val_mae: 0.0527 - 0.02s/epoch - 0.019s/step
Epoch 6/1000  
1/1 - p1 -  [*lr: 0.0009549] - loss: 0.0527 - mae: 0.0527 - val_loss: 0.0527 - val_mae: 0.0527 - 0.02s/epoch - 0.019s/step
Epoch 7/1000  
1/1 - p0 -  [*lr: 0.0034549] - loss: 0.0527 - mae: 0.0527 - val_loss: 0.0503 - val_mae: 0.0503 - 0.02s/epoch - 0.020s/step
Epoch 8/1000  
1/1 - p0 -  [*lr:

In [57]:
res = model.predict(test_data.shape[0], data=series)
inverse_transform_prediction(res, scaler)
print(prediction_metrics(test_data, res))
res

{'mae': 22620.614148727927, 'mape': 0.1550694888859028, 'wmape': 0.1722470534552511, 'quantile_acc': 0.18333333333333332}


,date,ta,ta_lower,ta_upper
0,2021-06-26,123773.295302,117095.409845,130451.180760
1,2021-06-27,116896.881664,112128.324223,121665.439105
2,2021-06-28,124292.357958,117470.347827,131114.368090
3,2021-06-29,115156.996820,110871.541626,119442.452014
4,2021-06-30,112572.590331,109004.730089,116140.450572
5,2021-07-01,112800.040353,109169.025578,116431.055129
6,2021-07-02,121916.872525,115754.447626,128079.297423
7,2021-07-03,126758.629854,119251.826417,134265.433292
8,2021-07-04,117286.868446,112410.025971,122163.710921
9,2021-07-05,111536.081966,108256.022105,114816.141828


In [58]:
test_load_and_save_model('time2vec.zip', model, scaler, data=series)

{'mae': 22620.614148727927, 'mape': 0.1550694888859028, 'wmape': 0.1722470534552511, 'quantile_acc': 0.18333333333333332}


,date,ta,ta_lower,ta_upper
0,2021-06-26,123773.295302,117095.409845,130451.180760
1,2021-06-27,116896.881664,112128.324223,121665.439105
2,2021-06-28,124292.357958,117470.347827,131114.368090
3,2021-06-29,115156.996820,110871.541626,119442.452014
4,2021-06-30,112572.590331,109004.730089,116140.450572
5,2021-07-01,112800.040353,109169.025578,116431.055129
6,2021-07-02,121916.872525,115754.447626,128079.297423
7,2021-07-03,126758.629854,119251.826417,134265.433292
8,2021-07-04,117286.868446,112410.025971,122163.710921
9,2021-07-05,111536.081966,108256.022105,114816.141828


## TransformerModel

In [59]:
model = TransformerModel(time_col=time_col, target_col=target_col, lags=lags, random_state=42, 
                               quantile=quantile, enable_progress_bar=False, enable_model_summary=False)

In [60]:
model.fit(data)

In [61]:
res = model.predict(test_data.shape[0], data=series)
inverse_transform_prediction(res, scaler)
print(prediction_metrics(test_data, res))
res

{'mae': 21564.10078125, 'mape': 0.15262893824069193, 'wmape': 0.1642021209309769, 'quantile_acc': 0.2}


,date,ta,ta_lower,ta_upper
0,2021-06-26,121139.359375,115285.179688,126993.539062
1,2021-06-27,121953.070312,115876.468750,128029.679688
2,2021-06-28,123176.757812,116765.656250,129587.859375
3,2021-06-29,123571.406250,117052.429688,130090.382812
4,2021-06-30,122295.945312,116125.609375,128466.281250
5,2021-07-01,121598.359375,115618.710938,127578.000000
6,2021-07-02,122971.398438,116616.421875,129326.367188
7,2021-07-03,121687.328125,115683.359375,127691.289062
8,2021-07-04,120410.437500,114755.507812,126065.367188
9,2021-07-05,120776.765625,115021.703125,126531.835938


In [62]:
test_load_and_save_model('transformer.zip', model, scaler, data=series)

{'mae': 21564.10078125, 'mape': 0.15262893824069193, 'wmape': 0.1642021209309769, 'quantile_acc': 0.2}


,date,ta,ta_lower,ta_upper
0,2021-06-26,121139.359375,115285.179688,126993.539062
1,2021-06-27,121953.070312,115876.468750,128029.679688
2,2021-06-28,123176.757812,116765.656250,129587.859375
3,2021-06-29,123571.406250,117052.429688,130090.382812
4,2021-06-30,122295.945312,116125.609375,128466.281250
5,2021-07-01,121598.359375,115618.710938,127578.000000
6,2021-07-02,122971.398438,116616.421875,129326.367188
7,2021-07-03,121687.328125,115683.359375,127691.289062
8,2021-07-04,120410.437500,114755.507812,126065.367188
9,2021-07-05,120776.765625,115021.703125,126531.835938


## TiDE

In [63]:
model = TiDEModel(time_col=time_col, target_col=target_col, lags=lags, random_state=42, 
                               quantile=0.9, enable_progress_bar=False, 
                 enable_model_summary=False)

In [64]:
model.fit(data)
res = model.predict(test_data.shape[0], data=series)
inverse_transform_prediction(res, scaler)
print(prediction_metrics(test_data, res))
res

{'mae': 24562.992447916666, 'mape': 0.16961599456248164, 'wmape': 0.18703749798212954, 'quantile_acc': 0.15}


,date,ta,ta_lower,ta_upper
0,2021-07-16,116203.945312,111789.359375,120618.531250
1,2021-07-17,112545.804688,109111.085938,115980.523438
2,2021-07-18,119353.906250,114095.570312,124612.234375
3,2021-07-19,118870.835938,113741.890625,123999.765625
4,2021-07-20,116981.382812,112358.554688,121604.218750
5,2021-07-21,116691.242188,112146.125000,121236.359375
6,2021-07-22,118679.320312,113601.687500,123756.968750
7,2021-07-23,117344.515625,112624.414062,122064.617188
8,2021-07-24,114411.593750,110477.109375,118346.085938
9,2021-07-25,114046.070312,110209.492188,117882.656250


In [65]:
test_load_and_save_model('tide.zip', model, scaler, data=series)

{'mae': 24562.992447916666, 'mape': 0.16961599456248164, 'wmape': 0.18703749798212954, 'quantile_acc': 0.15}


,date,ta,ta_lower,ta_upper
0,2021-07-16,116203.945312,111789.359375,120618.531250
1,2021-07-17,112545.804688,109111.085938,115980.523438
2,2021-07-18,119353.906250,114095.570312,124612.234375
3,2021-07-19,118870.835938,113741.890625,123999.765625
4,2021-07-20,116981.382812,112358.554688,121604.218750
5,2021-07-21,116691.242188,112146.125000,121236.359375
6,2021-07-22,118679.320312,113601.687500,123756.968750
7,2021-07-23,117344.515625,112624.414062,122064.617188
8,2021-07-24,114411.593750,110477.109375,118346.085938
9,2021-07-25,114046.070312,110209.492188,117882.656250


## SegRNN

In [66]:
model = SegRNNModel(time_col=time_col, target_col=target_col, lags=lags, random_state=42, 
                     quantile=quantile, accelerator=accelerator, patience=100, verbose=True)
model.fit(data)

MPS  available: True   | MPS  use: True
CUDA available: False  | CUDA use: False
TPU  available: False  | TPU  use: False
CPU  available: True   | CPU  use: False


Epoch 1/1000  
1/1 - p0 -  [*lr: 0.0009045] - loss: 0.1646 - mae: 0.1646 - val_loss: 0.1418 - val_mae: 0.1418 - 0.46s/epoch - 0.461s/step
Epoch 2/1000  
1/1 - p0 -  [*lr: 0.0006545] - loss: 0.1418 - mae: 0.1418 - val_loss: 0.1221 - val_mae: 0.1221 - 0.08s/epoch - 0.076s/step
Epoch 3/1000  
1/1 - p0 -  [*lr: 0.0003455] - loss: 0.1221 - mae: 0.1221 - val_loss: 0.1090 - val_mae: 0.1090 - 0.07s/epoch - 0.074s/step
Epoch 4/1000  
1/1 - p0 -  [*lr: 9.55e-05] - loss: 0.1090 - mae: 0.1090 - val_loss: 0.1024 - val_mae: 0.1024 - 0.08s/epoch - 0.082s/step
Epoch 5/1000  
1/1 - p0 - [*lr: 0.000000e+00] - loss: 0.1024 - mae: 0.1024 - val_loss: 0.1006 - val_mae: 0.1006 - 0.07s/epoch - 0.071s/step
Epoch 6/1000  
1/1 - p1 -  [*lr: 9.55e-05] - loss: 0.1006 - mae: 0.1006 - val_loss: 0.1006 - val_mae: 0.1006 - 0.07s/epoch - 0.066s/step
Epoch 7/1000  
1/1 - p0 -  [*lr: 0.0003455] - loss: 0.1006 - mae: 0.1006 - val_loss: 0.0989 - val_mae: 0.0989 - 0.07s/epoch - 0.071s/step
Epoch 8/1000  
1/1 - p0 -  [*lr: 0

In [67]:
res = model.predict(test_data.shape[0], data=series)
inverse_transform_prediction(res, scaler)
print(prediction_metrics(test_data, res))
res

{'mae': 21670.94869449536, 'mape': 0.15325281191547793, 'wmape': 0.1650157256414127, 'quantile_acc': 0.18333333333333332}


,date,ta,ta_lower,ta_upper
0,2021-06-26,120395.498841,115564.743981,125226.253701
1,2021-06-27,121891.780412,116711.374088,127072.186735
2,2021-06-28,122093.359265,116865.847943,127320.870587
3,2021-06-29,121289.435799,116249.785520,126329.086078
4,2021-06-30,121071.610371,116082.861594,126060.359147
5,2021-07-01,120974.964754,116008.800150,125941.129358
6,2021-07-02,121190.668065,116174.097855,126207.238274
7,2021-07-03,121731.554448,116588.589769,126874.519127
8,2021-07-04,121272.592759,116236.878366,126308.307152
9,2021-07-05,121570.805056,116465.404336,126676.205776


In [68]:
test_load_and_save_model('seg_rnn.zip', model, scaler, data=series)

{'mae': 21670.94869449536, 'mape': 0.15325281191547793, 'wmape': 0.1650157256414127, 'quantile_acc': 0.18333333333333332}


,date,ta,ta_lower,ta_upper
0,2021-06-26,120395.498841,115564.743981,125226.253701
1,2021-06-27,121891.780412,116711.374088,127072.186735
2,2021-06-28,122093.359265,116865.847943,127320.870587
3,2021-06-29,121289.435799,116249.785520,126329.086078
4,2021-06-30,121071.610371,116082.861594,126060.359147
5,2021-07-01,120974.964754,116008.800150,125941.129358
6,2021-07-02,121190.668065,116174.097855,126207.238274
7,2021-07-03,121731.554448,116588.589769,126874.519127
8,2021-07-04,121272.592759,116236.878366,126308.307152
9,2021-07-05,121570.805056,116465.404336,126676.205776


# MultiOutputRegressorModel

In [69]:
from PipelineTS.ml_model import MultiOutputRegressorModel

In [70]:
from xgboost import XGBRegressor

model = MultiOutputRegressorModel(time_col=time_col, target_col=target_col, lags=lags, random_state=42, 
                     quantile=quantile, estimator=XGBRegressor)
model.fit(data, fit_kwargs={'verbose': False})

In [71]:
res = model.predict(test_data.shape[0], data=series)
inverse_transform_prediction(res, scaler)
print(prediction_metrics(test_data, res))
res

{'mae': 21995.49841258948, 'mape': 0.15871965694394893, 'wmape': 0.1674870437176546, 'quantile_acc': 0.18333333333333332}


,date,ta,ta_lower,ta_upper
0,2021-06-26,150294.380623,136628.408002,163960.353244
1,2021-06-27,132702.161822,123790.158294,141614.165350
2,2021-06-28,144724.937814,132564.003260,156885.872368
3,2021-06-29,143798.676095,131888.046499,155709.305692
4,2021-06-30,153023.269641,138619.865648,167426.673634
5,2021-07-01,138549.137440,128057.097778,149041.177101
6,2021-07-02,132803.216006,123863.904469,141742.527543
7,2021-07-03,126200.792905,119045.663097,133355.922712
8,2021-07-04,130498.111434,122181.711425,138814.511443
9,2021-07-05,125432.693867,118485.128502,132380.259232


In [72]:
test_load_and_save_model('mor.zip', model, scaler, data=series)

{'mae': 21995.49841258948, 'mape': 0.15871965694394893, 'wmape': 0.1674870437176546, 'quantile_acc': 0.18333333333333332}


,date,ta,ta_lower,ta_upper
0,2021-06-26,150294.380623,136628.408002,163960.353244
1,2021-06-27,132702.161822,123790.158294,141614.165350
2,2021-06-28,144724.937814,132564.003260,156885.872368
3,2021-06-29,143798.676095,131888.046499,155709.305692
4,2021-06-30,153023.269641,138619.865648,167426.673634
5,2021-07-01,138549.137440,128057.097778,149041.177101
6,2021-07-02,132803.216006,123863.904469,141742.527543
7,2021-07-03,126200.792905,119045.663097,133355.922712
8,2021-07-04,130498.111434,122181.711425,138814.511443
9,2021-07-05,125432.693867,118485.128502,132380.259232


# MultiStepRegressorModel

In [73]:
from PipelineTS.ml_model.multi_output_model import MultiStepRegressorModel
from xgboost import XGBRegressor

model = MultiStepRegressorModel(time_col=time_col, target_col=target_col, lags=lags, random_state=42, 
                     quantile=quantile, estimator=XGBRegressor)
model.fit(data, fit_kwargs={'verbose': False})

In [74]:
res = model.predict(test_data.shape[0], data=series)
inverse_transform_prediction(res, scaler)
print(prediction_metrics(test_data, res))
res

{'mae': 22785.636620537938, 'mape': 0.1719889410380775, 'wmape': 0.17350363448069483, 'quantile_acc': 0.23333333333333334}


,date,ta,ta_lower,ta_upper
0,2021-06-26,150294.380623,134609.276062,165979.485183
1,2021-06-27,152125.037728,135872.140717,168377.934738
2,2021-06-28,137084.032332,125496.219396,148671.845269
3,2021-06-29,134893.637751,123985.192623,145802.082879
4,2021-06-30,151717.643901,135591.103235,167844.184568
5,2021-07-01,136083.102913,124805.735971,147360.469854
6,2021-07-02,136762.614700,125274.491926,148250.737474
7,2021-07-03,132616.656308,122414.434573,142818.878043
8,2021-07-04,129054.466900,119957.085741,138151.848058
9,2021-07-05,127056.829979,118579.031351,135534.628607


In [75]:
test_load_and_save_model('msr.zip', model, scaler, data=series)

{'mae': 22785.636620537938, 'mape': 0.1719889410380775, 'wmape': 0.17350363448069483, 'quantile_acc': 0.23333333333333334}


,date,ta,ta_lower,ta_upper
0,2021-06-26,150294.380623,134609.276062,165979.485183
1,2021-06-27,152125.037728,135872.140717,168377.934738
2,2021-06-28,137084.032332,125496.219396,148671.845269
3,2021-06-29,134893.637751,123985.192623,145802.082879
4,2021-06-30,151717.643901,135591.103235,167844.184568
5,2021-07-01,136083.102913,124805.735971,147360.469854
6,2021-07-02,136762.614700,125274.491926,148250.737474
7,2021-07-03,132616.656308,122414.434573,142818.878043
8,2021-07-04,129054.466900,119957.085741,138151.848058
9,2021-07-05,127056.829979,118579.031351,135534.628607
